# Old

In [5]:
function conv2d(img, kernel, stride = 1, padding = "valid")
    input_h, input_w = size(img)
    kernel_h, kernel_w = size(kernel)
    
    if padding == "same"
        pad_h = (kernel_h-1) ÷ 2
        pad_w = (kernel_w-1) ÷ 2
        
        img_padded = zeros(input_h+(2*pad_h),input_w+(2*pad_w))
        
        for i in 1:input_h , j in 1:input_w
            img_padded[i+pad_h, j+pad_w] = img[i,j]
        end
        
    elseif padding == "valid"
    else
        throw(DomainError(padding, "Invalid padding value"))
    end
    
    result = zeros((input_h- kernel_h) ÷ stride +1 ,(input_w- kernel_w) ÷ stride +1 )
    result_h, result_w = size(result)
    
    ih, iw = 1, 1
    for i in 1: result_h
        for j in 1: result_w
            for k in 1:kernel_h
                for l in 1:kernel_w
                    result[i,j] += img[ih+k-1, iw+l-1]*kernel[k,l]
                end
            end
            ih+=stride
        end
        iw+= stride
        ih = 1
    end
    return result
end

conv2d (generic function with 3 methods)

In [6]:
img = rand(Float32,10000,10000)
kernel = rand(Float32,15,15);

In [7]:
@time conv2d(img,kernel)

 53.943691 seconds (59.00 k allocations: 763.824 MiB, 0.08% gc time)


9986×9986 Array{Float64,2}:
 55.1103  53.6555  54.7989  55.7792  …  57.1378  56.0684  55.5278  54.8168
 55.638   55.1453  56.2829  57.517      56.9378  55.9692  55.291   56.6514
 59.5069  54.6726  56.5088  55.3175     57.3875  55.3549  54.2167  53.6572
 55.1441  55.0903  55.7647  55.8247     56.1902  54.3936  54.8783  55.4977
 58.0125  58.4033  60.4215  59.5335     56.3709  56.581   55.0934  57.8492
 59.7877  60.1004  61.0574  60.5322  …  56.0156  54.544   53.8752  53.9561
 60.4831  59.2864  61.8398  59.8047     55.0457  55.3872  55.5735  54.5301
 61.8257  59.6132  61.0708  62.2667     53.8662  55.0981  54.1424  55.8933
 59.7856  58.9797  60.4625  62.6842     56.3706  57.5737  57.2259  57.7376
 59.1531  61.1174  58.5238  60.0542     55.0528  53.5145  54.0603  55.2477
 60.5447  57.4912  60.7377  61.1334  …  53.444   55.6145  55.3125  55.3896
 58.5153  58.0574  59.0052  61.2728     56.0957  54.7779  52.5846  54.1418
 59.8011  59.6502  60.6901  59.6189     54.2842  55.3179  53.5531  53.12

# Loops P1

In [4]:
using FFTW

In [5]:
img = rand(1000,1000)
kernel = rand(15,15);

In [3]:
function conv2d_gpu(img, kernel, stride = 1, padding = "valid")
    input_h, input_w = size(img)
    kernel_h, kernel_w = size(kernel)
    
    if padding == "same"
        pad_h = (kernel_h-1) ÷ 2
        pad_w = (kernel_w-1) ÷ 2
        img_padded = zeros(input_h+(2*pad_h),input_w+(2*pad_w))
        for i in 1:input_h , j in 1:input_w
            img_padded[i+pad_h, j+pad_w] = img[i,j]
            img = img_padded
        end
    elseif padding == "valid"
    else
        throw(DomainError(padding, "Invalid padding value"))
    end
    @info size(img)
    result = zeros((input_h- kernel_h) ÷ stride +1 ,(input_w- kernel_w) ÷ stride +1 )
    result_h, result_w = size(result)
    
  ih, iw = 1, 1
    for i in 1: result_h
        for j in 1: result_w
#             for k in 1:kernel_h
#                 for l in 1:kernel_w
# #                     result[i,j] += img[ih+k-1, iw+l-1]*kernel[k,l]
                    
#                 end
#             end
             result[i,j] +=ifft(fft(img[ih+k-1, iw+l-1])*fft(kernel))
            ih+=stride
        end
        iw+= stride
        ih = 1
    end
    return result
end

conv2d_gpu (generic function with 3 methods)

In [4]:
# @time 
@time conv2d_gpu(img,kernel)

┌ Info: (1000, 1000)
└ @ Main In[3]:17


MethodError: MethodError: no method matching fft(::Float64)
Closest candidates are:
  fft(!Matched::AbstractArray{#s12,N} where N where #s12<:Real) at /home/subhaditya/.julia/packages/AbstractFFTs/mhQvY/src/definitions.jl:198
  fft(!Matched::AbstractArray{#s13,N} where N where #s13<:Real, !Matched::Any) at /home/subhaditya/.julia/packages/AbstractFFTs/mhQvY/src/definitions.jl:198
  fft(!Matched::AbstractArray{#s18,N} where N where #s18<:(Complex{#s19} where #s19<:Union{Integer, Rational})) at /home/subhaditya/.julia/packages/AbstractFFTs/mhQvY/src/definitions.jl:200
  ...

# CuDA


In [28]:
using CUDA,GPUArrays,CUDA.CUFFT

In [1]:
img = rand(10000,10000)
kernel = rand(15,15);

In [26]:
# CUBLAS.cublasLoggerConfigure(1, 0, 1, C_NULL)

In [29]:
CUDA.allowscalar(true)

In [2]:
using Images,ImageViewew

InitError: InitError: InterruptException:
during initialization of module CompilerSupportLibraries_jll

In [3]:
function conv2d_fft(img, kernel, stride = 1, padding = "valid")
    input_h, input_w = size(img)
    kernel_h, kernel_w = size(kernel)
    pad_h = (kernel_h-1) ÷ 2
    pad_w = (kernel_w-1) ÷ 2

    img_padded = zeros(input_h+(2*pad_h),input_w+(2*pad_w))
    em_kernel = zeros(size(img_padded))
    @info size(em_kernel)
    

    for i in 1:input_h , j in 1:input_w
        img_padded[i+pad_h, j+pad_w] = img[i,j]
    end
    imshow(img_padded)
#     @info size(img_padded), size(kernel)
    
    
end

conv2d_fft (generic function with 3 methods)

In [4]:
@time conv2d_fft(img,kernel)

┌ Info: (10014, 10014)
└ @ Main In[3]:9


UndefVarError: UndefVarError: imshow not defined

In [14]:
img_padded

UndefVarError: UndefVarError: img_padded not defined

# FFT

$f*g= ℱ^{-1}\big\{ℱ\{f\}\cdot ℱ\{g\}\big\}$

In [31]:
using CUDA.CUFFT

In [43]:
@time ifft(fft(img)*fft(img))

  0.001866 seconds (742 allocations: 23.516 KiB)


I! cuBLAS (v10.2) function cublasStatus_t cublasZgemm_v2(cublasContext*, cublasOperation_t, cublasOperation_t, int, int, int, const cuDoubleComplex*, const cuDoubleComplex*, int, const cuDoubleComplex*, int, const cuDoubleComplex*, cuDoubleComplex*, int) called:
i!  handle: type=cublasHandle_t; val=POINTER (IN HEX:0x0xb8ed650)
i!  transa: type=cublasOperation_t; val=CUBLAS_OP_N(0)
i!  transb: type=cublasOperation_t; val=CUBLAS_OP_N(0)
i!  m: type=int; val=1000
i!  n: type=int; val=1000
i!  k: type=int; val=1000
i!  alpha: type=SOME TYPE; val=POINTER (IN HEX:0x0x7ff4149185f0)
i!  A: type=SOME TYPE; val=POINTER (IN HEX:0x0x7ff111000000)
i!  lda: type=int; val=1000
i!  B: type=SOME TYPE; val=POINTER (IN HEX:0x0x7ff116000000)
i!  ldb: type=int; val=1000
i!  beta: type=SOME TYPE; val=POINTER (IN HEX:0x0x7ff414918650)
i!  C: type=SOME TYPE; val=POINTER (IN HEX:0x0x7ff11d000000)
i!  ldc: type=int; val=1000
i! Time: 2020-07-03T20:32:01 elapsed from start 9.666667 minutes or 580.000000 seconds


1000×1000 CuArray{Complex{Float64},2,Nothing}:
 243012.0-2.21743e-12im  248462.0+4.01465e-12im  …  241187.0-2.52501e-12im
 246895.0+3.073e-12im    252705.0+6.76299e-13im     246505.0+2.79914e-13im
 244040.0+1.29822e-12im  248105.0-5.81032e-13im     241536.0-1.12949e-12im
 243674.0+2.3766e-12im   252479.0-7.71853e-12im     241838.0+2.15367e-12im
 245296.0-8.24491e-12im  252462.0+1.12814e-12im     246401.0+5.69911e-13im
 242099.0-6.79953e-12im  252635.0-4.2406e-12im   …  242305.0+4.35187e-12im
 238863.0+3.01047e-12im  250017.0+1.15065e-13im     239439.0+1.7488e-12im
 235779.0-5.5786e-12im   244164.0-1.35589e-12im     228264.0-4.2733e-12im
 251754.0+2.63911e-12im  255426.0+1.13083e-12im     244159.0+5.11367e-13im
 246463.0-3.65313e-13im  255684.0-3.11646e-12im     243004.0+3.5881e-12im
 254619.0+1.5321e-12im   255587.0+3.03261e-12im  …  249885.0+4.20335e-12im
 245245.0+3.32796e-12im  256673.0+1.39922e-12im     250151.0+1.76742e-12im
 255243.0+1.23851e-12im  265632.0-2.16189e-12im     2574